##DATASET CREATION

In [ ]:
import pandas as pd

# We load the given datasets
df1 = pd.read_csv('/content/data_X.csv')  # Per-minute data from sensors (temperature, moisture, thickness)
df2 = pd.read_csv('/content/data_Y.csv')  # Hourly data from quality testing

# We convert the date_time columns to datetime format
df1['date_time'] = pd.to_datetime(df1['date_time'])
df2['date_time'] = pd.to_datetime(df2['date_time'])

# We filter for temperature data starting from one hour before the first quality measurement
# This is to ensure that we only take the relevant sensor data for each quality score, considering that the product stays in the oven for 1 hour
start_date = df2['date_time'].min() - pd.Timedelta(hours=1)
df1 = df1[df1['date_time'] >= start_date]

# We now we average the sensor data over the preceding hour for every quality measurement
# This is to ensure that we consider the mean chamber temperature during the one hour that the product is in the oven
# We do this by setting the date_time as index for df1 and calculating hourly means for all columns (from hh:00 to hh:59)
# hh:00 to hh:59 is considered since the humidity is the same for these times for every set of 60 records (1 hour)
# From the above, we infer that it is the same product/batch since it is only measured ONCE, before entering the oven
# The 5-minute delay to quality measure (hh+1:05) is considered to be the time required for QC
df1.set_index('date_time', inplace=True)
df1_hourly = df1.resample('H').mean().reset_index()  # Aggregating all columns

# We adjust the timestamp to match the quality reading time (hh+1:05)
df1_hourly['date_time'] = df1_hourly['date_time'] + pd.Timedelta(hours=1, minutes=5)

# We perform an inner join on date_time to align with quality data
merged_df = pd.merge(df1_hourly, df2, on='date_time', how='inner')

# We save the merged dataset for modeling
merged_df.to_csv('data_merged_mean.csv', index=False)
print(merged_df.head())

            date_time  T_data_1_1  T_data_1_2  T_data_1_3  T_data_2_1  \
0 2015-01-04 00:05:00  271.745455  338.545455  267.218182  328.236364   
1 2015-01-04 01:05:00  277.583333  300.366667  273.550000  320.483333   
2 2015-01-04 02:05:00  273.600000  231.833333  266.800000  322.700000   
3 2015-01-04 03:05:00  250.333333  227.033333  256.350000  326.583333   
4 2015-01-04 04:05:00  240.400000  239.350000  249.250000  325.750000   

   T_data_2_2  T_data_2_3  T_data_3_1  T_data_3_2  T_data_3_3  T_data_4_1  \
0  329.400000  345.472727  501.981818  499.400000  588.636364  320.490909   
1  331.966667  355.450000  501.900000  501.650000  705.516667  330.233333   
2  334.216667  347.133333  501.133333  500.366667  579.600000  341.550000   
3  333.666667  317.716667  511.183333  498.116667  492.366667  345.350000   
4  325.400000  310.500000  522.683333  498.966667  538.716667  341.283333   

   T_data_4_2  T_data_4_3  T_data_5_1  T_data_5_2  T_data_5_3      H_data  \
0  362.181818  336.36

##DATA PREPARATION

In [ ]:
# We import all relevant packages required for modeling with RF, AdaBoost, LR & SVC
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from imblearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # We filter out warnings that specific features will be discontinued in future

# We load the merged dataset
dataset = pd.read_csv("/content/data_merged_mean.csv")
pd.set_option('display.max_columns', None)
print(dataset.head())
print(dataset.shape)
print(dataset.info())
print(dataset.describe())

# We handle date-time
# We extract useful features from it, in case the time, day or date could be important for prediction
dataset['date_time'] = pd.to_datetime(dataset['date_time'])
print(dataset.info())
final_data = dataset
final_data['year'] = final_data['date_time'].dt.year
final_data['month'] = final_data['date_time'].dt.month
final_data['day'] = final_data['date_time'].dt.day
final_data['hour'] = final_data['date_time'].dt.hour
print(final_data.head(2))

# We redefine 'quality' as a categorical variable so that it can be used for binary classification
# Here the quality is set to positive class (high quality) if it is greater than or equal to 410
# This is to ensure that we eliminate false negatives (low quality products that are predicted as high quality) by maximizing PRECISION
final_data['quality_class'] = np.where(final_data['quality'] >= 410, 1, 0)

# # Correlation matrix
# plt.figure(figsize=(10, 6))
# corr_matrix = final_data.corr()
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix of Features')
# plt.show()

# We drop the old 'quality' column containing numerical quality measurements
final_data = final_data.drop('quality', axis=1)

print(final_data.info())
print(final_data.head(2))

# We redefine X and Y
X = final_data.drop(['quality_class', 'date_time'], axis=1)
Y = final_data['quality_class']

print(X.shape)
print(Y.shape)

feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

print(np.any(np.isnan(X_scaled)))  # We check for NaNs
print(np.any(np.isinf(X_scaled)))  # We check for infinite values

print(Y.value_counts()) # We count the number of high & low quality instances for cost-benefit analysis

             date_time  T_data_1_1  T_data_1_2  T_data_1_3  T_data_2_1  \
0  2015-01-04 00:05:00  271.745455  338.545455  267.218182  328.236364   
1  2015-01-04 01:05:00  277.583333  300.366667  273.550000  320.483333   
2  2015-01-04 02:05:00  273.600000  231.833333  266.800000  322.700000   
3  2015-01-04 03:05:00  250.333333  227.033333  256.350000  326.583333   
4  2015-01-04 04:05:00  240.400000  239.350000  249.250000  325.750000   

   T_data_2_2  T_data_2_3  T_data_3_1  T_data_3_2  T_data_3_3  T_data_4_1  \
0  329.400000  345.472727  501.981818  499.400000  588.636364  320.490909   
1  331.966667  355.450000  501.900000  501.650000  705.516667  330.233333   
2  334.216667  347.133333  501.133333  500.366667  579.600000  341.550000   
3  333.666667  317.716667  511.183333  498.116667  492.366667  345.350000   
4  325.400000  310.500000  522.683333  498.966667  538.716667  341.283333   

   T_data_4_2  T_data_4_3  T_data_5_1  T_data_5_2  T_data_5_3      H_data  \
0  362.181818  

## RANDOM FOREST

### Random Forest with ALL Features

In [ ]:
from sklearn.metrics import recall_score

# We implement the Random Forest model beginning with no. of estimators: [50,100,200,500,800]
# This is then tuned repeatedly until we get the best parameter

model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=1) )
    ])
grid_param = {'classification__n_estimators': [7,9,10,11,13]}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

gd_sr.fit(X_scaled, Y)

# We save the best parameters and results for comparison

best_parameters_rf = gd_sr.best_params_
print(best_parameters_rf)

best_result_rf = gd_sr.best_score_
print("Precision: ", best_result_rf)

# We calculate and print recall score for the best model (for cost-benefit analysis)
best_model = gd_sr.best_estimator_  # Get the best model from GridSearchCV
Y_pred = best_model.predict(X_scaled)  # Make predictions on the full dataset
recall = recall_score(Y, Y_pred)  # Calculate recall score
print("Recall:", recall)

# We print the feature importance list to run new models with the top features

featimp = pd.Series(gd_sr.best_estimator_.named_steps["classification"].feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importance list for the best model
print(featimp)

{'classification__n_estimators': 10}
Precision:  0.9334954621569473
Recall: 0.9947545111204364
T_data_3_3    0.302721
T_data_3_2    0.201301
T_data_3_1    0.143297
H_data        0.050298
T_data_5_2    0.036001
T_data_5_1    0.032217
T_data_1_2    0.028095
T_data_1_1    0.027054
T_data_5_3    0.026127
T_data_1_3    0.023558
T_data_2_3    0.019436
T_data_2_2    0.018442
T_data_2_1    0.015259
AH_data       0.012924
T_data_4_3    0.011590
T_data_4_2    0.011427
T_data_4_1    0.010681
day           0.009308
hour          0.008735
month         0.007713
year          0.003817
dtype: float64


### Random Forest with TOP 3 Features

In [ ]:
# The top 3 features as given by RF are temperature data for chamber 3
# We select features with higher significance and redefine the feature set
X_3 = final_data[['T_data_3_2', 'T_data_3_3', 'T_data_3_1']]

# We scale the data so that it is all given equal weightage
feature_scaler = StandardScaler()
X_scaled_3 = feature_scaler.fit_transform(X_3)

# We tune the random forest parameter 'n_estimators' beginnig from [10,50,100,200,300] and refine repeatedly
# We also implement cross-validation using Grid Search
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=1) )
    ])
grid_param = {'classification__n_estimators': [7,9,10,11,13]}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

gd_sr.fit(X_scaled_3, Y)

# We save the best parameters and results for comparison

best_parameters_rf3 = gd_sr.best_params_
print(best_parameters_rf3)

best_result_rf3 = gd_sr.best_score_
print("Precision: ", best_result_rf3)

{'classification__n_estimators': 10}
Precision:  0.8605403366630145


### Random Forest with TOP 4 Features

In [ ]:
# The top 4 features as given by RF are temperature data for chamber 3, and the height of input product
# We select features with higher significance and redefine the feature set
X_4 = final_data[['T_data_3_2', 'T_data_3_3', 'T_data_3_1', 'H_data']]

# We scale the data so that it is all given equal weightage
feature_scaler = StandardScaler()
X_scaled_4 = feature_scaler.fit_transform(X_4)

# We tune the random forest parameter 'n_estimators' beginnig from [10,50,100,200,300] and refine repeatedly
# We also implement cross-validation using Grid Search
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=1) )
    ])
grid_param = {'classification__n_estimators': [7,9,10,11,13]}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

gd_sr.fit(X_scaled_4, Y)

# We save the best parameters and results for comparison

best_parameters_rf4 = gd_sr.best_params_
print(best_parameters_rf4)

best_result_rf4 = gd_sr.best_score_
print("Precision: ", best_result_rf4)

{'classification__n_estimators': 10}
Precision:  0.8785821364753211


## ADABOOST

In [ ]:
# We implement the Random Forest model beginning with no. of estimators: [50,100,200,500,800]
# This is then tuned repeatedly until we get the best parameter

model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', AdaBoostClassifier(random_state=1))
    ])
grid_param = {'classification__n_estimators': [240,250,260,270]}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

gd_sr.fit(X_scaled, Y)

# We save the best parameters and results for comparison

best_parameters_adb = gd_sr.best_params_
print(best_parameters_adb)

best_result_adb = gd_sr.best_score_
print("Precision: ", best_result_adb)

# We print the feature importance list to run new models with the top features

featimp = pd.Series(gd_sr.best_estimator_.named_steps["classification"].feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importance list for the best model
print(featimp)

{'classification__n_estimators': 250}
Precision:  0.9217329846977884
T_data_3_3    0.124
T_data_3_2    0.120
T_data_3_1    0.112
H_data        0.072
T_data_5_1    0.068
T_data_1_2    0.068
T_data_2_3    0.060
T_data_1_1    0.056
T_data_5_3    0.052
T_data_5_2    0.048
T_data_2_1    0.040
T_data_4_2    0.028
T_data_2_2    0.028
T_data_1_3    0.024
T_data_4_3    0.020
day           0.020
T_data_4_1    0.016
AH_data       0.016
hour          0.016
month         0.012
year          0.000
dtype: float64


## LOGISTIC REGRESSION

In [ ]:
# We implement Logistic Regression using a Stochastic Gradient Descent Classifier
# We tune eta0 to control the rate at which the model learns - a lower learning rate implies smaller increments/steps
# We tune max_iter to set the upper limit on how many times the model passes over the training data to reach convergence
# We tune alpha to control the strictness of regularization, i.e. the amount of penalty applied to larger coefficients to prevent overfitting
# We tune the l1 ratio in elasticnet to control the proportion of lasso (L1) vs. ridge (L2) regularization applied
# We implement cross-validation using Grid Search
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', SGDClassifier(loss = 'log_loss', penalty = 'elasticnet', random_state = 1))
    ])
grid_param = {'classification__eta0': [.000001,.000003,.000005,.000007,.00001], 'classification__max_iter' : [20,25,30,35], 'classification__alpha': [.0005,.0001,.005,.001], 'classification__l1_ratio': [0.1,0.5,0.7,0.9,1]}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

gd_sr.fit(X_scaled, Y)

# We save the best parameters and results for comparison

best_parameters_lr = gd_sr.best_params_
print(best_parameters_lr)

best_result_lr = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Precision: ", best_result_lr)

{'classification__alpha': 0.0001, 'classification__eta0': 1e-06, 'classification__l1_ratio': 1, 'classification__max_iter': 30}
Precision:  0.9192557053152702


## SUPPORT VECTOR CLASSIFIER

In [ ]:
# We tune the kernel to map the datapoints to the best-fitting higher dimension space
# We tune the regularization parameter C to allow for wider/narrower margin
# Larger C implies less regularization, i.e. a narrower margin and stricter classification, which may lead to a more complex model with overfitting

model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', SVC(random_state=1) )
    ])
grid_param = {'classification__kernel': ['linear','poly','rbf','sigmoid'], 'classification__C': [.001,.01,.05,.01]}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

gd_sr.fit(X_scaled, Y)

# We save the best parameters and results for comparison

best_parameters_svc = gd_sr.best_params_
print(best_parameters_svc)

best_result_svc = gd_sr.best_score_
print("Precision: ", best_result_svc)

## RESULTS

In [ ]:
print("Precision Random Forest:       ", best_result_rf)
print("Precision Random Forest Top 3: ", best_result_rf3)
print("Precision Random Forest Top 4: ", best_result_rf4)
print("Precision AdaBoost:            ", best_result_adb)
print("Precision Logistic Regression: ", best_result_lr)
print("Precision Support Vector:      ", best_result_svc)

Since the product is a premium/high-end food product, we aim to minimize false positives. This is because a low quality product that is mistakenly sent into market will damage the reputation of the brand. This is unacceptable since quality cannot be compromised in a premium product, and the ripple effects among the customer base will be catastrophic.
Comparatively, sacrificing a proportion of mis-classified high quality products is not as big an issue, since only the cost of production + QC + opportunity cost is lost. This is a small price to pay to preserve brand value.

The best precision is obtained from a Random Forest Classifier utilizing all the features (columns).
We observe the precision reducing significantly when choosing just the top features. Hence, all the input values are of some importance in prediction, despite the top three features (temperatures of chamber 3) having relatively a far greater impact.